# Determination of ratio D in Forward Field Calculation using Variable Density Tesseroids

## Hypotesis

We are going to start with certain assumptions. We will supose that the density is a linear function on height:

$\rho(\Delta h) = a\Delta h + b$

Lets compute the value of the gravity potential generated by a Tesseroid with such density by approximating it with a mesh of Tesseroids with constant densities. Each value will be the one that the density function assumes at the center of this little Tesseroids.

In order to do this we must know beforehand the shape of this mesh. Because the variation is only in the radius axis, the mesh can have no divisions in the lon lat axis, so only the radius axis will be divided. Although, a high value for the distance-size ratio D will be used, in order to increase precission in the calculation.

We can find out the optimal number of divisions in the radius axis ($n_r$) by calculating the potential generated by mesh with different values of $n_r$. For lower values of $n_r$, the potential values corresponding with $n_r$ and $n_r -1$ will be very different. But for higher $n_r$, the potential of the mesh will be closer to the true value.

In order to obtain this "true" value, we will calculate the potential for increasing values of $n_r$ until the maximum difference between the potentials created with $n_r$ and $n_r - 1$ is smaller than $10^{-9}\%$.

**Import Packages**

In [1]:
from __future__ import division
import time
import numpy as np
from tesseroid_density import tesseroid
from fatiando.mesher import Tesseroid, TesseroidMesh
from fatiando import gridder
import matplotlib.pyplot as plt

%matplotlib inline

**Definition of linear density function**

In [2]:
def density_linear(height):
    return height

**Creation of the computation grid**

In [3]:
area = (-30, 30, -30, 30)
shape = (20, 20)
h = 10e3
lons, lats, heights = gridder.regular(area, shape, z=h)

**Bounds and D ratio of the Tesseroid Mesh**

In [ ]:
D_high = 8
w, e, s, n, top, bottom = -10, 10, -10, 10, -3000, -50000

**Determination of optimal tesseroid mesh size**

In [ ]:
"Initialization of the loop"
potential_single_point = []
nr_values = []
nr = 0
true_potential = 1e50*np.ones(lons.size)
while True:
    nr += 1
    mesh_shape = (nr, 1, 1)
    model_mesh = TesseroidMesh((w, e, s, n, top, bottom), mesh_shape)
    
    density_list = []
    for tess in model_mesh:
        lon1, lon2, lat1, lat2, r1, r2 = tess.get_bounds()
        density_list.append(density_linear(0.5*(r1 + r2)))
    model_mesh.addprop("density", density_list)
    old = true_potential[:]
    true_potential = getattr(tesseroid, 'potential')(lons,
                                                     lats,
                                                     heights,
                                                     model_mesh,
                                                     ratio=D_high)
    potential_single_point.append(true_potential[0])
    nr_values.append(nr)
    diff = 100*np.max(np.abs((true_potential-old)/old))
    if diff < 1e-9:
        break

potential_single_point = np.array(potential_single_point)
nr_optimal = nr
print "Optimal nr: ", nr_optimal
mesh_shape = (nr_optimal, 1, 1)

**Plot**

In [ ]:
plt.plot(nr_values, potential_single_point, '-', marker='.')
plt.show()

plt.contourf(lons.reshape(shape), lats.reshape(shape), true_potential.reshape(shape), 150)
plt.colorbar()
plt.axis('equal')
plt.show()

## Determination of distance-size ratio D for Tesseroid with variable density

Now we use the tesseroid with variable density to calculate the potential using several distance-size ratios and evaluate the maximum difference between them and the "true" potential. We will take the best value of D as the lower one that is below the $10^{-1}\%$ threshold.

In [ ]:
# Calculate error for each distance-size ratio D
D_values = np.arange(0.5, 4.5, 0.5)
errors = []
model = Tesseroid(w, e, s, n, top, bottom, props={"density":density_linear})
for D in D_values:
    st = time.time()
    result = getattr(tesseroid, 'potential')(lons, lats, heights, [model], ratio=D)
    print D, "time: ", time.time() - st, "s"
    diff = 100*np.max(np.abs((result - true_potential)/true_potential))
    errors.append(diff)

**Plot D values vs Maximum difference**

In [ ]:
plt.plot(D_values, errors, 'o-')
plt.plot([D_values[0], D_values[-1]], [1e-1, 1e-1], '-')
plt.semilogy()
plt.show()